# Descarga de datos WeatherBench2
---

``python version = 3.11``

En este notebook se exploran los conjuntos de datos de WeatherBench2

In [6]:
# !pip install apache-beam
# !pip install git+https://github.com/google-research/weatherbench2.git
# !pip install gcsfs
# !pip install netCDF4 

In [7]:
import apache_beam  
import weatherbench2
import numpy as np
import xarray as xr

## ERA 5, resolución: 1440x721, niveles: 13, time step: 6 H
---

> **Nomenclatura:** ``1959-2023_01_10-wb13-6h-1440x721_with_derived_variables`` 
> * fecha inicial: ``1959``
> * fecha final: ``2023_01_10``
> * niveles: ``wb13``
> * delta tiempo: ``6h``
> * resolución espacial: ``1440x721``
> * info. extra: ``with_derived_variables`` 

In [11]:
data_UHR = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr')
data_UHR

<xarray.Dataset>
Dimensions:                                           (time: 93544,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 13)
Coordinates:
  * latitude                                          (latitude) float32 90.0...
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float32 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/62)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 ...
    10m_v_component_of_wind                           (time, latitude, longitude) float32 ...
    10m_wind_speed                                    (time, latitude, longitude) float32 ...
    2m_dewpoint_temperature                           (time, latitude, longitude) float32 ...
    2m_temperature                                    (time, latitude, longitude) float32 ...
    above_ground                                      (time, level, latitude, longitude) float32 ...
    ...                                                ...
    volumetric_soil_water_layer_1                     (time, latitude, longitude) float32 ...
    volumetric_soil_water_layer_2                     (time, latitude, longitude) float32 ...
    volumetric_soil_water_layer_3                     (time, latitude, longitude) float32 ...
    volumetric_soil_water_layer_4                     (time, latitude, longitude) float32 ...
    vorticity                                         (time, level, latitude, longitude) float32 ...
    wind_speed                                        (time, level, latitude, longitude) float32 ...

In [28]:
# Peso en GB
data_UHR.nbytes / 1000**3

88963.00301054

In [19]:
data_UHR.to_netcdf(r"C:\Users\gcuervo\Documents\Doctorado\DB\1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.nc")

MemoryError: Unable to allocate 362. GiB for an array with shape (93544, 721, 1440) and data type float32

La base de datos pesa ``88.963 GB`` por lo que no es posible guardarla en local.

### Selección de variables
---
Debido a que la base de datos es demasiado grande se intenta seleccionar solo unas cuantas variables ver si se puede descargar en local.

In [34]:
data_sel_vars = data_UHR[['geopotential', 'temperature', 'specific_humidity', 'u_component_of_wind', 'v_component_of_wind']]
data_sel_vars

<xarray.Dataset>
Dimensions:              (time: 93544, level: 13, latitude: 721, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 50 100 150 200 250 ... 700 850 925 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * time                 (time) datetime64[ns] 1959-01-01 ... 2023-01-10T18:0...
Data variables:
    geopotential         (time, level, latitude, longitude) float32 ...
    temperature          (time, level, latitude, longitude) float32 ...
    specific_humidity    (time, level, latitude, longitude) float32 ...
    u_component_of_wind  (time, level, latitude, longitude) float32 ...
    v_component_of_wind  (time, level, latitude, longitude) float32 ...

In [36]:
# Peso en GB
data_sel_vars.nbytes / 1000**3

25251.4926227

In [37]:
data_sel_vars.to_netcdf(r"C:\Users\gcuervo\Documents\Doctorado\DB\1959-2023_01_10-wb13-6h-1440x721_with_geop_temp_spehum_uvwind.nc")


MemoryError: Unable to allocate 4.59 TiB for an array with shape (93544, 13, 721, 1440) and data type float32

De la base de datos total se seleccionaron 5 variables: ``'geopotential', 'temperature', 'specific_humidity', 'u_component_of_wind', 'v_component_of_wind'``
Con un peso de: ``25251.5 GB``

## ERA 5, resolución: 512x256, niveles: 13, time step: 6 H
---

In [38]:
data_HR = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2022-6h-512x256_equiangular_conservative.zarr')
data_HR

<xarray.Dataset>
Dimensions:                                           (time: 92044,
                                                       longitude: 512,
                                                       latitude: 256, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 -89....
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float64 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 ...
    10m_v_component_of_wind                           (time, longitude, latitude) float32 ...
    10m_wind_speed                                    (time, longitude, latitude) float32 ...
    2m_temperature                                    (time, longitude, latitude) float32 ...
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 ...
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 ...
    ...                                                ...
    type_of_high_vegetation                           (longitude, latitude) float32 ...
    type_of_low_vegetation                            (longitude, latitude) float32 ...
    u_component_of_wind                               (time, level, longitude, latitude) float32 ...
    v_component_of_wind                               (time, level, longitude, latitude) float32 ...
    vertical_velocity                                 (time, level, longitude, latitude) float32 ...
    wind_speed                                        (time, level, longitude, latitude) float32 ...

In [39]:
# Peso en GB
data_HR.nbytes / 1000**3

5211.825067208

## ERA 5, resolución: 240x121, niveles: 13, time step: 6 H
---

In [54]:
data_MR = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-240x121_equiangular_with_poles_conservative.zarr')
data_MR

<xarray.Dataset>
Dimensions:                                           (time: 93544,
                                                       longitude: 240,
                                                       latitude: 121, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 -90....
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float64 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/62)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 ...
    10m_v_component_of_wind                           (time, longitude, latitude) float32 ...
    10m_wind_speed                                    (time, longitude, latitude) float32 ...
    2m_dewpoint_temperature                           (time, longitude, latitude) float32 ...
    2m_temperature                                    (time, longitude, latitude) float32 ...
    above_ground                                      (time, level, longitude, latitude) float32 ...
    ...                                                ...
    volumetric_soil_water_layer_1                     (time, longitude, latitude) float32 ...
    volumetric_soil_water_layer_2                     (time, longitude, latitude) float32 ...
    volumetric_soil_water_layer_3                     (time, longitude, latitude) float32 ...
    volumetric_soil_water_layer_4                     (time, longitude, latitude) float32 ...
    vorticity                                         (time, level, longitude, latitude) float32 ...
    wind_speed                                        (time, level, longitude, latitude) float32 ...

In [55]:
# Peso en GB
data_MR.nbytes / 1000**3

2488.332529584

## ERA 5, resolución: 128x64, niveles: 13, time step: 6 H
---

In [56]:
data_LR = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2022-6h-128x64_equiangular_conservative.zarr')
data_LR

<xarray.Dataset>
Dimensions:                                           (time: 92044,
                                                       longitude: 128,
                                                       latitude: 64, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 -88....
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float64 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 ...
    10m_v_component_of_wind                           (time, longitude, latitude) float32 ...
    10m_wind_speed                                    (time, longitude, latitude) float32 ...
    2m_temperature                                    (time, longitude, latitude) float32 ...
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 ...
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 ...
    ...                                                ...
    type_of_high_vegetation                           (longitude, latitude) float32 ...
    type_of_low_vegetation                            (longitude, latitude) float32 ...
    u_component_of_wind                               (time, level, longitude, latitude) float32 ...
    v_component_of_wind                               (time, level, longitude, latitude) float32 ...
    vertical_velocity                                 (time, level, longitude, latitude) float32 ...
    wind_speed                                        (time, level, longitude, latitude) float32 ...

In [57]:
# Peso en GB
data_LR.nbytes / 1000**3

325.73975828

## ERA 5, resolución: 64x32, niveles: 13, time step: 6 H
---

In [58]:
data_ULR = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-64x32_equiangular_conservative.zarr')
data_ULR

<xarray.Dataset>
Dimensions:                                           (time: 93544,
                                                       longitude: 64,
                                                       latitude: 32, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 -87....
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float64 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/62)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 ...
    10m_v_component_of_wind                           (time, longitude, latitude) float32 ...
    10m_wind_speed                                    (time, longitude, latitude) float32 ...
    2m_dewpoint_temperature                           (time, longitude, latitude) float32 ...
    2m_temperature                                    (time, longitude, latitude) float32 ...
    above_ground                                      (time, level, longitude, latitude) float32 ...
    ...                                                ...
    volumetric_soil_water_layer_1                     (time, longitude, latitude) float32 ...
    volumetric_soil_water_layer_2                     (time, longitude, latitude) float32 ...
    volumetric_soil_water_layer_3                     (time, longitude, latitude) float32 ...
    volumetric_soil_water_layer_4                     (time, longitude, latitude) float32 ...
    vorticity                                         (time, level, longitude, latitude) float32 ...
    wind_speed                                        (time, level, longitude, latitude) float32 ...

In [59]:
# Peso en GB
data_ULR.nbytes / 1000**3

175.486406312